In [ ]:
%cd drive/MyDrive/project

/content/drive/MyDrive/project


In [ ]:
import torch
import numpy as np
import random

seed = 42
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [ ]:
!pip install transformers
!pip install vncorenlp
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.6 MB/s 
     |████████████████████████████████| 120 kB 73.6 MB/s 
     |████████████████████████████████| 6.6 MB 78.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 14.7 MB/s 
  Created wheel for vncorenlp: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=227544a9f0d1424a3f8c673eb11fde25c99f2f6573d250182eed53c33e55c749
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built vncorenlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 14.9 MB/s 
     |████████████████████████████████| 115 kB 87.1 MB/s 
     |████████████████████████████████| 212 kB 70.6 MB/s 
   

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

# phobert = AutoModel.from_pretrained("vinai/phobert-large")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
import re
def strip_emoji(text):
    RE_EMOJI = re.compile(u'([\U00002600-\U000027BF])|([\U0001f300-\U0001f64F])|([\U0001f680-\U0001f6FF])')
    return RE_EMOJI.sub(r'', text)

def remove_special_char(text):
  special_character = re.compile("�+")
  return special_character.sub(r'', text)

def remove_punctuation(text):
  punctuation = re.compile(r"[!#$%&()*+;<=>?@[\]^_`{|}~]")
  return punctuation.sub(r"", text)

def remove_number(text):
  return re.sub(" \d+", " ", text)

def normalize_annotatation(text):
  khach_san = "\bkhach san ?|\bksan ?|\bks ?"
  return re.sub("\bnv ?", "nhân viên",re.sub(khach_san, "khách sạn", text))

In [ ]:
import re

In [ ]:
from datasets import load_dataset

data_files = {'train': "data/data_full_add_nha_hang_submit.csv", 'test': "data/test_data_submit.csv"}
dataset = load_dataset('csv', data_files=data_files)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-bd0e7f1fd67e1bc1/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
def clean_text(example):
  return {"Review": normalize_annotatation(remove_number(remove_special_char(remove_punctuation(strip_emoji(example["Review"].lower())))))}
def segment(example):
  return {"Segment": " ".join([" ".join(sen) for sen in rdrsegmenter.tokenize(example["Review"])])}

dataset = dataset.map(clean_text)
dataset = dataset.map(segment)

  0%|          | 0/5121 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

  0%|          | 0/5121 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [ ]:
import numpy as np

feature = ['giai_tri', 'luu_tru', 'nha_hang', 'an_uong', 'di_chuyen', 'mua_sam']

def tokenize(example):
  return tokenizer(example["Segment"],truncation=True)

def label(example):
  return {'labels_regressor': np.array([example[i] for i in feature]),
          'labels_classifier': np.array([int(example[i] != 0) for i in feature])}

dataset = dataset.map(tokenize, batched=True)
dataset = dataset.map(label)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/5121 [00:00<?, ?ex/s]

  0%|          | 0/500 [00:00<?, ?ex/s]

In [ ]:
tokenized_datasets = dataset.remove_columns(['Unnamed: 0' ,'Review', 'giai_tri', 'luu_tru', 'nha_hang', 'an_uong', 'di_chuyen', 'mua_sam', 'Segment'])
tokenized_datasets.set_format("torch")

In [ ]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=32, collate_fn=data_collator
)
test_dataloader = DataLoader(
    tokenized_datasets["test"], batch_size=32, collate_fn=data_collator
)

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModelSoftmax(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelSoftmax, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768*4, 6)
    self.regressor = nn.Linear(768*4, 30)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
    outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    outputs_regressor = outputs_regressor.reshape(-1, 6, 5)
    # outputs = self.output2(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    # outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_classifier, outputs_regressor

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModelSoftmax_v2(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelSoftmax_v2, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.regressor = nn.Linear(768*4, 36)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
    outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_regressor = self.regressor(outputs)
    outputs_regressor = outputs_regressor.reshape(-1, 6, 6)
    # outputs = self.output2(outputs)
    # outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_regressor

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModel(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModel, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768*4, 6)
    self.regressor = nn.Linear(768*4, 6)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
    outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    # outputs = self.output2(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_classifier, outputs_regressor

In [ ]:
import math
import torch.nn as nn
from transformers import AutoModel, AutoConfig

class MyCustomClassifierLayer(nn.Module):
    def __init__(self, size_in, size_out):
        super().__init__()
        self.size_in, self.size_out = size_in, size_out
        weights = torch.Tensor(size_in, size_out)
        self.weights = nn.Parameter(weights)  # nn.Parameter is a Tensor that's a module parameter.
        # bias = torch.Tensor(size_out)
        # self.bias = nn.Parameter(bias)

        # initialize weights and biases
        nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5)) # weight init
        # fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weights)
        # bound = 1 / math.sqrt(fan_in)
        # nn.init.uniform_(self.bias, -bound, bound)  # bias init

    def forward(self, x):
        # w_times_x= torch.mm(x, self.weights.t())
        # return torch.add(w_times_x, self.bias)  # w times x + b
        return torch.mul(x, self.weights).sum(dim=-1)

class MyCustomRegressorLayer(nn.Module):
  def __init__(self, size1, size2, size3):
    super().__init__()
    self.size1, self.size2, self.size3 = size1, size2, size3
    weights = torch.Tensor(size1, size2, size3)
    self.weights = nn.Parameter(weights)
    nn.init.kaiming_uniform_(self.weights, a=math.sqrt(5))
  def forward(self, x):
    # x : N, 6, 768
    x = x.unsqueeze(-2).repeat(1, 1, 5, 1)
    return torch.mul(x, self.weights).sum(dim=-1)

class CustomModelLabelAttention(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelLabelAttention, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.reduce = nn.Linear(768, 6)
    # self.flatten = nn.Flatten()
    # self.classifier = nn.Linear(768, 6)
    # self.regressor = nn.Linear(768, 30)
    self.classifier = MyCustomClassifierLayer(6, 768)
    self.regressor = MyCustomRegressorLayer(6, 5, 768)
    # self.encoderlayer = EncoderLayer(768, 64, 64, 12, 3096)
    # self.ff = nn.Linear(768, 768)
    # self.norm1 = nn.LayerNorm(768)
    # self.norm2 = nn.LayerNorm(768)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    # outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
    outputs = self.dropout(outputs[0])
    embedding = outputs # N, S, 768
    outputs = self.reduce(outputs) # N, S, 6
    outputs = outputs.transpose(1, 2) # N, 6, S
    outputs = nn.Softmax(dim=-1)(outputs) # N, 6, S
    attention_embed = torch.matmul(outputs, embedding) # N, 6, 768
    # outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_classifier = self.classifier(attention_embed)
    outputs_regressor = self.regressor(attention_embed)
    # outputs = self.output2(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    # outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_classifier, outputs_regressor

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModelEnsemble(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelEnsemble, self).__init__()
    self.num_outputs = num_outputs
    self.aspects = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"]
    self.models = nn.ModuleList([AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True)) 
     for i in range(len(self.aspects))])
    for model in self.models:
      for parameter in model.parameters():
        parameter.require_grad = False
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    # self.dropout = nn.Dropout(0.1)
    self.classifiers = nn.ModuleList([nn.Linear(768*4, 1) for i in range(len(self.aspects))])
    self.regressors = nn.ModuleList([nn.Linear(768*4, 1) for i in range(len(self.aspects))])
    # self.classifier = nn.Linear(768*4, 6)
    # self.regressor = nn.Linear(768*4, 6)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs_classifiers = []
    outputs_regressors = []
    for i in range(len(self.aspects)):
      outputs = self.models[i](input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
      outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
      # outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
      outputs_classifier = self.classifiers[i](outputs)
      outputs_regressor = self.regressors[i](outputs)
    # outputs = self.output2(outputs)
      outputs_classifier = nn.Sigmoid()(outputs_classifier) # N, 1
      outputs_regressor = nn.Sigmoid()(outputs_regressor)*5 # N, 1

      outputs_classifiers.append(outputs_classifier)
      outputs_regressors.append(outputs_regressor)
    outputs_classifiers = torch.cat(outputs_classifiers, dim=-1) # N, 6
    outputs_regressors = torch.cat(outputs_regressors, dim=-1) # N, 6
    return outputs_classifiers, outputs_regressors

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig
class CustomModelOneAspect(nn.Module):
  def __init__(self, checkpoint, aspect):
    super(CustomModelOneAspect, self).__init__()
    self.aspect = aspect
    self.model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    # self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768*4, 1)
    self.regressor = nn.Linear(768*4, 1)
    # self.classifier = nn.Linear(768*4, 6)
    # self.regressor = nn.Linear(768*4, 6)
    # self.output2 = nn.Linear(96, 6)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    # outputs = self.dropout(outputs_sequence[2])
    outputs = torch.cat((outputs[2][-1][:,0, ...],outputs[2][-2][:,0, ...], outputs[2][-3][:,0, ...], outputs[2][-4][:,0, ...]),-1)
      # outputs = self.dropout(outputs)
    # outputs = self.output1(outputs[:, 0, :])
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    # outputs = self.output2(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier) # N, 1
    outputs_regressor = nn.Sigmoid()(outputs_regressor)*5 # N, 1
    return outputs_classifier, outputs_regressor

In [ ]:
import torch.nn as nn
from transformers import AutoModel, AutoConfig


class ScaledDotProductAttention(nn.Module):
    def __init__(self, d_k):
        super(ScaledDotProductAttention, self).__init__()
        self.d_k = d_k

    def forward(self, Q, K, V, attn_mask):
        # print(Q.shape, K.shape, V.shape, attn_mask.shape)
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(self.d_k) # scores : [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)
        context = torch.matmul(attn, V)
        return context, attn

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, d_k,d_v, n_heads):
        super(MultiHeadAttention, self).__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.linear = nn.Linear(n_heads * d_v, d_model)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size x len_q x d_model], k: [batch_size x len_k x d_model], v: [batch_size x len_k x d_model]
        residual, batch_size = Q, Q.size(0)
        # (B, S, D) -proj-> (B, S, D) -split-> (B, S, H, W) -trans-> (B, H, S, W)
        q_s = self.W_Q(Q).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # q_s: [batch_size x n_heads x len_q x d_k]
        k_s = self.W_K(K).view(batch_size, -1, self.n_heads, self.d_k).transpose(1,2)  # k_s: [batch_size x n_heads x len_k x d_k]
        v_s = self.W_V(V).view(batch_size, -1, self.n_heads, self.d_v).transpose(1,2)  # v_s: [batch_size x n_heads x len_k x d_v]
        
        attn_mask = attn_mask.unsqueeze(1).repeat(1, self.n_heads, 1)
        attn_mask = attn_mask.unsqueeze(-2).repeat(1, 1, Q.shape[1], 1) # attn_mask : [batch_size x n_heads x len_q x len_k]
        # print(attn_mask.shape)
        # context: [batch_size x n_heads x len_q x d_v], attn: [batch_size x n_heads x len_q(=len_k) x len_k(=len_q)]
        context, attn = ScaledDotProductAttention(self.d_k)(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.n_heads * self.d_v) # context: [batch_size x len_q x n_heads * d_v]
        output = self.linear(context)
        return self.layer_norm(output + residual) # output: [batch_size x len_q x d_model]

class PoswiseFeedForwardNet(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1)
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.layer_norm = nn.LayerNorm(d_model)

    def forward(self, inputs):
        residual = inputs # inputs : [batch_size, len_q, d_model]
        output = nn.ReLU()(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
        return self.layer_norm(output + residual)

class EncoderLayer(nn.Module):
    def __init__(self, d_model, d_k, d_v, n_heads, d_ff):
        super(EncoderLayer, self).__init__()
        self.d_model = d_model
        self.d_k = d_k
        self.d_v = d_v
        self.n_heads = n_heads
        self.d_ff = d_ff
        self.enc_self_attn = MultiHeadAttention(d_model, d_k,d_v, n_heads)
        self.pos_ffn = PoswiseFeedForwardNet(d_model, d_ff)

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) # enc_inputs to same Q,K,V
        enc_outputs = self.pos_ffn(enc_outputs) # enc_outputs: [batch_size x len_q x d_model]
        return enc_outputs

class CustomModelMultiHead(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelMultiHead, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768, 6)
    self.regressor = nn.Linear(768, 30)
    self.encoderlayer = EncoderLayer(768, 64, 64, 12, 3096)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    outputs = self.dropout(outputs[0])
    outputs = self.encoderlayer(outputs, attention_mask.type(torch.bool)) # N, S, D
    outputs = outputs[:, 0, :]
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    outputs_regressor = outputs_regressor.reshape(-1, 6, 5)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    return outputs_classifier, outputs_regressor

class CustomModelMultiHeadRegressor(nn.Module):
  def __init__(self, checkpoint, num_outputs):
    super(CustomModelMultiHeadRegressor, self).__init__()
    self.num_outputs = num_outputs
    self.model = model = AutoModel.from_pretrained(checkpoint,config=AutoConfig.from_pretrained(checkpoint, output_attentions=True,output_hidden_states=True))
    # for parameter in self.model.parameters():
    #   parameter.require_grad = False
    self.dropout = nn.Dropout(0.1)
    self.classifier = nn.Linear(768, 6)
    self.regressor = nn.Linear(768, 6)
    self.encoderlayer = EncoderLayer(768, 64, 64, 12, 3096)
  def forward(self, input_ids=None, attention_mask=None):
    outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
    outputs = self.dropout(outputs[0])
    outputs = self.encoderlayer(outputs, attention_mask.type(torch.bool)) # N, S, D
    outputs = outputs[:, 0, :]
    outputs_classifier = self.classifier(outputs)
    outputs_regressor = self.regressor(outputs)
    outputs_classifier = nn.Sigmoid()(outputs_classifier)
    outputs_regressor = nn.Sigmoid()(outputs_regressor)*5
    return outputs_classifier, outputs_regressor

In [ ]:
import torch.nn.functional as F


def loss_classifier(pred_classifier, labels_classifier):
  return nn.BCELoss()(pred_classifier, labels_classifier)


def loss_regressor(pred_regressor, labels_regressor):
  mask = (labels_regressor != 0)
  loss = ((pred_regressor - labels_regressor)**2)[mask].sum() / mask.sum()
  return loss

def loss_softmax(inputs, labels, device):
  mask = (labels != 0)
  # inputs (N, 6, 5)
  n, aspect, rate = inputs.shape
  num = 0
  loss = torch.zeros(labels.shape).to(device)
  for i in range(aspect):
    # print(inputs[:, i, :].shape, labels[:, i].shape)
    label_i = labels[:, i].clone()
    label_i[label_i != 0] -= 1
    label_i = label_i.type(torch.LongTensor).to(device)
    loss[:, i] = nn.CrossEntropyLoss(reduction='none')(inputs[:, i, :], label_i)
  loss = loss[mask].sum() / mask.sum()
  return loss

def loss_softmax_v2(inputs, labels, device):
  # inputs N, 6, 6
  # labels N, 6
  loss = torch.zeros(labels.shape).to(device)
  aspects = labels.shape[1]
  for i in range(aspects):
    label_i = labels[:, i].clone()
    label_i = label_i.type(torch.LongTensor).to(device)
    loss[:, i] = nn.CrossEntropyLoss(reduction="none")(inputs[:, i, :], label_i)
  loss = loss.mean()
  return loss

def sigmoid_focal_loss(
    inputs: torch.Tensor,
    targets: torch.Tensor,
    alpha: float = 0.25,
    gamma: float = 2,
    reduction: str = "none",):

    # p = torch.sigmoid(inputs)
    p = inputs
    ce_loss = F.binary_cross_entropy(inputs, targets, reduction="none")
    # ce_loss = nn.BCELoss(reduce=False)(inputs, targets)
    p_t = p * targets + (1 - p) * (1 - targets)
    loss = ce_loss * ((1 - p_t) ** gamma)

    if alpha >= 0:
        alpha_t = alpha * targets + (1 - alpha) * (1 - targets)
        loss = alpha_t * loss

    if reduction == "mean":
        loss = loss.mean()
    elif reduction == "sum":
        loss = loss.sum()

    return loss

def bce_loss_weights(inputs, targets, weights):
  ce_loss = F.binary_cross_entropy(inputs, targets, reduction="none")
  weights = targets*(1 / weights.view(1, -1)) + (1 - targets)*(1 / (1 - weights.view(1, -1)))
  loss = ce_loss*weights
  return loss.mean()


def CB_loss(inputs, targets, samples_positive_per_cls, samples_negative_per_cls, no_of_classes=2,loss_type='sigmoid', beta=0.9999, gamma=2):
  # print(targets.shape)
  # samples_positive_per_cls = targets.sum(dim=0) # num_cls
  # samples_negative_per_cls = len(targets) - samples_positive_per_cls # num_cls
  samples_per_cls = torch.concat([samples_positive_per_cls.unsqueeze(-1), samples_negative_per_cls.unsqueeze(-1)], dim=-1) # num_cls, 2
  # print(samples_per_cls.shape)
  effective_num = 1.0 - torch.pow(beta, samples_per_cls) # num_cls, 2
  weights = (1.0 - beta) / effective_num # num_cls, 2
  # print(weights.shape)
  weights = weights / weights.sum(dim=-1).reshape(-1, 1) * no_of_classes # num_cls, 2 
  weights = targets*weights[:, 0] + (1 - targets)*weights[:, 1]

  if loss_type == "focal":
    cb_loss = (sigmoid_focal_loss(inputs, targets)*weights).mean()
  elif loss_type == "sigmoid":
    cb_loss = (F.binary_cross_entropy(inputs,targets, reduction="none")*weights).mean()

  return cb_loss

In [ ]:
# model = CustomModelSoftmax("vinai/phobert-base", 30)
# model = CustomModel("vinai/phobert-base", 30)
# model = CustomModelLabelAttention("vinai/phobert-base", 30)
# model = CustomModelEnsemble("vinai/phobert-base", 30)
# model = CustomModelMultiHead("vinai/phobert-base", 30)
model = CustomModelMultiHeadRegressor("vinai/phobert-base", 30)
# model = CustomModelSoftmax_v2("vinai/phobert-base", 30)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
batch = next(iter(train_dataloader))
{k: v.shape for k, v in batch.items()}

{'input_ids': torch.Size([32, 133]),
 'token_type_ids': torch.Size([32, 133]),
 'attention_mask': torch.Size([32, 133]),
 'labels_regressor': torch.Size([32, 6]),
 'labels_classifier': torch.Size([32, 6])}

In [ ]:
# batch

In [ ]:
def unknown_token(inputs, device):
  augment_inputs = {k: v.clone() for k,v in inputs.items()}
  inputs_ids = augment_inputs["input_ids"]
  mask = augment_inputs["attention_mask"]
  mask_bos = (inputs_ids != 0)
  mask_eos = (inputs_ids != 2)
  mask = torch.logical_and(mask, torch.logical_and(mask_bos, mask_eos))
  proba = torch.rand(inputs_ids.shape).to(device)
  mask_proba = (proba < 0.1)
  final_mask = torch.logical_and(mask, mask_proba)
  inputs_ids[final_mask] = 3
  return augment_inputs

In [ ]:
# augment_inputs = unknown_token(batch)
# augment_inputs

In [ ]:
# batch

In [ ]:
# torch.set_printoptions(profile="full")

In [ ]:
# batch["input_ids"]

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model.to(device)

CustomModelMultiHeadRegressor(
  (model): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(64001, 768, padding_idx=1)
      (position_embeddings): Embedding(258, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm):

In [ ]:
from transformers import AdamW, get_scheduler

optimizer = AdamW(model.parameters(), lr=5e-5)
num_epochs = 10
num_training_steps = num_epochs*len(train_dataloader)
lr_scheduler = get_scheduler(
    'linear',
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
def pred_to_label(outputs_classifier, outputs_regressor):
  result = np.zeros((outputs_classifier.shape[0], 6))
  mask = (outputs_classifier >= 0.5)
  result[mask] = outputs_regressor[mask]
  return result

In [ ]:
def pred_to_label_one_aspect(outputs_classifier, outputs_regressor):
  result = np.zeros((outputs_classifier.shape[0], 1))
  mask = (outputs_classifier >= 0.5)
  result[mask] = outputs_regressor[mask]
  return result

In [ ]:
class ScalarMetric():
  def __init__(self):
    self.scalar = 0
    self.num = 0
  def update(self, scalar):
    self.scalar += scalar
    self.num += 1
    return self
  def compute(self):
    return self.scalar / self.num
  def reset(self):
    self.scalar = 0
    self.num = 0

class AccuracyMetric():
  def __init__(self):
    self.correct = 0
    self.num = 0
  def update(self, y_pred, y_true):
    self.correct += (y_pred == y_true).sum()
    self.num += len(y_pred)*y_pred.shape[1]
  def compute(self):
    return self.correct / self.num
  def reset(self):
    self.correct = 0
    self.num = 0

def precision(y_pred, y_true):
  true_positive = np.logical_and(y_pred, y_true).sum(axis=0)
  false_positive = np.logical_and(y_pred, np.logical_not(y_true)).sum(axis=0)
  return true_positive / (true_positive + false_positive)

def recall(y_pred, y_true):
  true_positive = np.logical_and(y_pred, y_true).sum(axis=0)
  false_negative = np.logical_and(np.logical_not(y_pred), y_true).sum(axis=0)
  return true_positive / (true_positive + false_negative)

class F1_score():
  def __init__(self):
    self.y_pred = None
    self.y_true = None
  def update(self, y_pred, y_true):
    self.y_pred = np.concatenate([self.y_pred, y_pred], axis=0) if self.y_pred is not None else y_pred
    self.y_true = np.concatenate([self.y_true, y_true], axis=0) if self.y_true is not None else y_true
  def compute(self):
    # print(self.y_pred != 0)
    # print(self.y_true != 0)
    f1_score = np.zeros(self.y_pred.shape[1])
    precision_score = precision(self.y_pred != 0, self.y_true != 0)
    recall_score = recall(self.y_pred != 0, self.y_true != 0)
    mask_precision_score = np.logical_and(precision_score != 0, np.logical_not(np.isnan(precision_score)))
    mask_recall_score = np.logical_and(recall_score != 0, np.logical_not(np.isnan(recall_score)))
    mask = np.logical_and(mask_precision_score, mask_recall_score)
    print("Precision:",precision_score)
    print("Recall", recall_score)
    f1_score[mask] = 2* (precision_score[mask] * recall_score[mask]) / (precision_score[mask] + recall_score[mask])
    return f1_score

class R2_score():
  def __init__(self):
    self.y_pred = None
    self.y_true = None
  def update(self, y_pred, y_true):
    self.y_pred = np.concatenate([self.y_pred, y_pred], axis=0) if self.y_pred is not None else y_pred
    self.y_true = np.concatenate([self.y_true, y_true], axis=0) if self.y_true is not None else y_true
  def compute(self):
    mask = np.logical_and(self.y_pred !=0, self.y_true != 0)
    rss = (((self.y_pred - self.y_true)**2)*mask).sum(axis=0) 
    k = (mask*16).sum(axis=0)
    r2_score = np.ones(rss.shape[0])
    mask2 = (k != 0)
    r2_score[mask2] = 1 - rss[mask2]/k[mask2]
    return r2_score

In [ ]:
# Eval
from tqdm.auto import tqdm

pb_train = tqdm(range(num_training_steps))
pb_test = tqdm(range(num_epochs*len(test_dataloader)))
best_score = -1

for epoch in range(num_epochs):
  train_loss = 0
  model.train()
  for batch in train_dataloader:
    inputs = {'input_ids': batch['input_ids'].to(device),
             'attention_mask': batch['attention_mask'].to(device)}
    outputs_classifier, outputs_regressor = model(**inputs)
    # loss = nn.MSELoss()(outputs, batch['labels'].to(device).float())
    loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
    # loss1 = sigmoid_focal_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), alpha=-2, gamma=1,reduction='mean')
    # loss1 = bce_loss_weights(outputs_classifier, batch['labels_classifier'].to(device).float(), weights.to(device))
    # loss1 = CB_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), num_positive.to(device), num_negative.to(device))
    loss2 = loss_regressor(outputs_regressor, batch['labels_regressor'].to(device).float())
    loss = 10*loss1 + loss2
    optimizer.zero_grad()
    # print(loss.item())
    loss.backward()
    optimizer.step()       
    lr_scheduler.step()
    pb_train.update(1)
    pb_train.set_postfix(loss_classifier=loss1.item(),loss_regressor=loss2.item(),loss=loss.item())
    train_loss += loss.item() / len(train_dataloader)
  print("Train Loss:", train_loss)

  val_loss = ScalarMetric()
  val_loss_classifier = ScalarMetric()
  val_loss_regressor = ScalarMetric()
  val_acc = AccuracyMetric()
  val_f1_score = F1_score()
  val_r2_score = R2_score()
  num = 0
  correct = 0
  result = None
  model.eval()
  for batch in test_dataloader:
    inputs = {'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)}
    with torch.no_grad():
      outputs_classifier, outputs_regressor = model(**inputs)
    # outputs = nn.Sigmoid()(outputs)
    # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
      loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
      loss2 = loss_regressor(outputs_regressor, batch['labels_regressor'].to(device).float())
      loss = loss1 + loss2
      outputs_classifier = outputs_classifier.cpu().numpy()
      outputs_regressor = outputs_regressor.cpu().numpy()
      y_true = batch['labels_regressor'].numpy()
      outputs = pred_to_label(outputs_classifier, outputs_regressor)
    # y_true = prob_to_label(y_true)
    # print(outputs)
      result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
      val_loss_classifier.update(loss1.item())
      val_loss_regressor.update(loss2.item())
      val_loss.update(loss.item())
      val_acc.update(np.round(outputs), y_true)
      val_f1_score.update(np.round(outputs), y_true)
      val_r2_score.update(np.round(outputs), y_true)
      pb_test.update(1)
  f1_score = val_f1_score.compute()
  r2_score = val_r2_score.compute()
  final_score = (f1_score * r2_score).sum()*1/6
  if final_score > best_score:
    best_score = final_score
    torch.save(model.state_dict(), 'weights/model_attention_regress_final_submit.pt')
  
  print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
  print("Acc", val_acc.compute())
  print("F1_score", f1_score)
  print("R2_score", r2_score)
  print("Final_score", final_score)
  print("Best_Score", best_score)

  0%|          | 0/1610 [00:00<?, ?it/s]

  0%|          | 0/160 [00:00<?, ?it/s]

Train Loss: 3.2591985752863923
Precision: [0.57692308 0.94444444 0.69230769 0.90740741 0.86       0.75609756]
Recall [0.6        0.66666667 0.45       0.93779904 0.82692308 0.73809524]
Test Loss: 0.9492856673896313 Loss Classifier: 0.2148975757881999 Loss Regressor: 0.7343880869448185
Acc 0.855
F1_score [0.58823529 0.7816092  0.54545455 0.92235294 0.84313725 0.74698795]
R2_score [0.95       0.96323529 0.97916667 0.9403699  0.96438953 0.91532258]
Final_score 0.701664771292435
Best_Score 0.701664771292435
Train Loss: 1.7230819871892096
Precision: [0.6031746  0.90697674 0.66666667 0.89908257 0.92045455 0.77777778]
Recall [0.50666667 0.76470588 0.45       0.93779904 0.77884615 0.66666667]
Test Loss: 0.7901366837322712 Loss Classifier: 0.19553521554917097 Loss Regressor: 0.5946014691144228
Acc 0.868
F1_score [0.55072464 0.82978723 0.53731343 0.91803279 0.84375    0.71794872]
R2_score [0.95065789 0.95352564 0.96875    0.96875    0.95910494 0.94642857]
Final_score 0.7022287917336049
Best_Scor

In [ ]:
# # Ensemble Aspects
# from tqdm.auto import tqdm
# from transformers import AdamW, get_scheduler

# aspects = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"]
# for index, aspect in enumerate(aspects):
#   model = CustomModelOneAspect("vinai/phobert-base", aspect)
#   device = "cuda" if torch.cuda.is_available() else "cpu"
#   model.to(device)

#   optimizer = AdamW(model.parameters(), lr=5e-5)
#   num_epochs = 10
#   num_training_steps = num_epochs*len(train_dataloader)
#   lr_scheduler = get_scheduler(
#     'linear',
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps
# )

#   pb_train = tqdm(range(num_training_steps))
#   pb_test = tqdm(range(num_epochs*len(test_dataloader)))
#   best_score = -1

#   for epoch in range(num_epochs):
#     train_loss = 0
#     model.train()
#     for batch in train_dataloader:
#       inputs = {'input_ids': batch['input_ids'].to(device),
#              'attention_mask': batch['attention_mask'].to(device)}
#       outputs_classifier, outputs_regressor = model(**inputs)
#     # loss = nn.MSELoss()(outputs, batch['labels'].to(device).float())
#     # labels_classifier shape : N, 6
#     # labels_regressors shape : N, 6
#       loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'][:, index:index+1].to(device).float())
#     # loss1 = sigmoid_focal_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), alpha=-2, gamma=1,reduction='mean')
#     # loss1 = bce_loss_weights(outputs_classifier, batch['labels_classifier'].to(device).float(), weights.to(device))
#     # loss1 = CB_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), num_positive.to(device), num_negative.to(device))
#       loss2 = loss_regressor(outputs_regressor, batch['labels_regressor'][:, index:index+1].to(device).float())
#       loss = 10*loss1 + loss2 if not torch.isnan(loss2) else 10*loss1
#       if torch.isnan(loss):
#         print("Loss Classifier", loss1)
#         print("Output Classifier", outputs_classifier)
#         print("Label Classifier", batch['labels_classifier'][:, index:index+1])
#         print("Loss Regressor", loss2)
#         print("Output Regressor", outputs_regressor)
#         print("Label Regressor", batch['labels_regressor'][:, index:index+1])
#       optimizer.zero_grad()
#     # print(loss.item())
#       loss.backward()
#       optimizer.step()       
#       lr_scheduler.step()
#       pb_train.update(1)
#       pb_train.set_postfix(loss_classifier=loss1.item(),loss_regressor=loss2.item(),loss=loss.item())
#       train_loss += loss.item() / len(train_dataloader)
#     print("Train Loss:", train_loss)

#     val_loss = ScalarMetric()
#     val_loss_classifier = ScalarMetric()
#     val_loss_regressor = ScalarMetric()
#     val_acc = AccuracyMetric()
#     val_f1_score = F1_score()
#     val_r2_score = R2_score()
#     num = 0
#     correct = 0
#     result = None
#     model.eval()
#     for batch in test_dataloader:
#       inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#       with torch.no_grad():
#         outputs_classifier, outputs_regressor = model(**inputs)
#     # outputs = nn.Sigmoid()(outputs)
#     # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
#         loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'][:, index:index+1].to(device).float())
#         loss2 = loss_regressor(outputs_regressor, batch['labels_regressor'][:, index:index+1].to(device).float())
#         loss = loss1 + loss2
#         outputs_classifier = outputs_classifier.cpu().numpy()
#         outputs_regressor = outputs_regressor.cpu().numpy()
#         y_true = batch['labels_regressor'].numpy()
#         outputs = pred_to_label_one_aspect(outputs_classifier, outputs_regressor)
#     # y_true = prob_to_label(y_true)
#     # print(outputs)
#         result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#         val_loss_classifier.update(loss1.item())
#         val_loss_regressor.update(loss2.item())
#         val_loss.update(loss.item())
#         val_acc.update(np.round(outputs), y_true)
#         val_f1_score.update(np.round(outputs), y_true[:, index:index+1])
#         val_r2_score.update(np.round(outputs), y_true[:, index:index+1])
#         pb_test.update(1)
#     f1_score = val_f1_score.compute()
#     r2_score = val_r2_score.compute()
#     final_score = (f1_score * r2_score).mean()
#     if final_score > best_score:
#       best_score = final_score
#       torch.save(model.state_dict(), f'weights/model_regress_{aspect}.pt')
  
#     print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
#     print("Acc", val_acc.compute())
#     print("F1_score", f1_score)
#     print("R2_score", r2_score)
#     print("Final_score", final_score)
#     print("Best_Score", best_score)

In [ ]:
# # Ensemble

# import numpy as np
# aspects = ["giai_tri", "luu_tru", "nha_hang", "an_uong","di_chuyen", "mua_sam"]
# models = []
# for aspect in aspects:
#   model = CustomModelOneAspect("vinai/phobert-base", aspect).to(device)
#   model.load_state_dict(torch.load(f"weights/model_regress_{aspect}.pt"))
#   model.eval()
#   models.append(model)
  
# # model1 = CustomModelSoftmax("vinai/phobert-base", 30).to(device)
# # model2 = CustomModel("vinai/phobert-base", 30).to(device)
# # model3 = CustomModelLabelAttention("vinai/phobert-base", 30).to(device)
# # model1.load_state_dict(torch.load("weights/model_softmax_v2_submit.pt"))
# # model2.load_state_dict(torch.load("weights/model_regress_v2_submit.pt"))
# # model3.load_state_dict(torch.load("weights/model_label_v2_submit.pt"))
# val_loss = ScalarMetric()
# val_loss_classifier = ScalarMetric()
# val_loss_regressor = ScalarMetric()
# val_acc = AccuracyMetric()
# val_f1_score = F1_score()
# val_r2_score = R2_score()
# num = 0
# correct = 0
# result = None
# # model1.eval()
# # model2.eval()
# for batch in test_dataloader:
#   inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#   with torch.no_grad():
#     outputs_classifiers = np.zeros((inputs["input_ids"].shape[0], len(aspects)))
#     outputs_regressors = np.zeros((inputs["input_ids"].shape[0], len(aspects)))
#     for i,model in enumerate(models):
#       outputs_classifier, outputs_regressor = model(**inputs)
#       # outputs_classifier2, outputs_regressor2 = model2(**inputs)
#     # outputs_classifier3, outputs_regressor3 = model3(**inputs)

#       outputs_classifier = outputs_classifier.cpu().numpy()
#       outputs_regressor = outputs_regressor.cpu().numpy()
#       outputs_classifiers[:, i:i+1] = outputs_classifier
#       outputs_regressors[:, i:i+1] = outputs_regressor
#     # outputs_classifier2 = outputs_classifier2.cpu().numpy()
#     # outputs_regressor2 = outputs_regressor2.cpu().numpy()
#     # outputs_classifier3 = outputs_classifier3.cpu().numpy()
#     # outputs_regressor3 = outputs_regressor3.cpu().numpy()
#     # outputs_regressor2 = score_to_tensor(outputs_regressor2)

#     # outputs_regressor = (nn.Softmax(dim=-1)(torch.tensor(outputs_regressor1)).numpy() + outputs_regressor2)/2
#     # outputs_classifier = (outputs_classifier1 + outputs_classifier2)/2

#     # outputs_regressor = outputs_regressor.argmax(axis=-1) + 1

#     y_true = batch['labels_regressor'].numpy()
#     outputs = pred_to_label(outputs_classifiers, outputs_regressors)
#     # y_true = prob_to_label(y_true)
#     # print(outputs)
#     result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#     val_loss_classifier.update(loss1.item())
#     val_loss_regressor.update(loss2.item())
#     val_loss.update(loss.item())
#     val_acc.update(np.round(outputs), y_true)
#     val_f1_score.update(np.round(outputs), y_true)
#     val_r2_score.update(np.round(outputs), y_true)
# f1_score = val_f1_score.compute()
# r2_score = val_r2_score.compute()
# final_score = (f1_score * r2_score).sum()*1/6
# print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
# print("Acc", val_acc.compute())
# print("F1_score", f1_score)
# print("R2_score", r2_score)
# print("Final_score", final_score)

In [ ]:
# outputs_classifiers.shape, outputs_regressors.shape

In [ ]:
# # Softmax
# from tqdm.auto import tqdm

# pb_train = tqdm(range(num_training_steps))
# pb_test = tqdm(range(num_epochs*len(test_dataloader)))
# best_score = -1

# for epoch in range(num_epochs):
#   train_loss = 0
#   val_loss = 0
#   model.train()
#   for batch in train_dataloader:
#     inputs = {'input_ids': batch['input_ids'].to(device),
#              'attention_mask': batch['attention_mask'].to(device)}
#     # outputs_classifier, outputs_regressor = model(**unknown_token(inputs, device))
#     outputs_classifier, outputs_regressor = model(**inputs)
#     # loss = nn.MSELoss()(outputs, batch['labels'].to(device).float())
#     # loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
#     loss1 = sigmoid_focal_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), alpha=-1, gamma=1,reduction='mean')
#     # loss1 = bce_loss_weights(outputs_classifier, batch['labels_classifier'].to(device).float(), weights.to(device))
#     # loss1 = CB_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), num_positive.to(device), num_negative.to(device))
    
#     loss2 = loss_softmax(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
#     loss = 10*loss1 + loss2
#     optimizer.zero_grad()
#     # print(loss.item())
#     loss.backward()
#     optimizer.step()       
#     lr_scheduler.step()
#     pb_train.update(1)
#     pb_train.set_postfix(loss_classifier=loss1.item(),loss_regressor=loss2.item(),loss=loss.item())
#     train_loss += loss.item() / len(train_dataloader)
#   print("Train Loss:", train_loss)
#   model.eval()
#   val_loss = ScalarMetric()
#   val_loss_classifier = ScalarMetric()
#   val_loss_regressor = ScalarMetric()
#   val_acc = AccuracyMetric()
#   val_f1_score = F1_score()
#   val_r2_score = R2_score()
#   num = 0
#   correct = 0
#   result = None
#   model.eval()
#   for batch in test_dataloader:
#     inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#     with torch.no_grad():
#       outputs_classifier, outputs_regressor = model(**inputs)
#       # outputs = nn.Sigmoid()(outputs)
#       # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
#       loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
#       loss2 = loss_softmax(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
#       loss = loss1 + loss2
#       outputs_classifier = outputs_classifier.cpu().numpy()
#       outputs_regressor = outputs_regressor.cpu().numpy()
#       outputs_regressor = outputs_regressor.argmax(axis=-1) + 1
#       y_true = batch['labels_regressor'].numpy()
#       outputs = pred_to_label(outputs_classifier, outputs_regressor)
#       # y_true = prob_to_label(y_true)
#       # print(outputs)
#       result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#       val_loss_classifier.update(loss1.item())
#       val_loss_regressor.update(loss2.item())
#       val_loss.update(loss.item())
#       val_acc.update(np.round(outputs), y_true)
#       val_f1_score.update(np.round(outputs), y_true)
#       val_r2_score.update(np.round(outputs), y_true)
#       pb_test.update(1)
#   f1_score = val_f1_score.compute()
#   r2_score = val_r2_score.compute()
#   final_score = (f1_score * r2_score).sum()*1/6
#   if final_score > best_score:
#     best_score = final_score
#     torch.save(model.state_dict(), "weights/model_attention_softmax_final_submit.pt")
#   print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
#   print("Acc", val_acc.compute())
#   print("F1_score", f1_score)
#   print("R2_score", r2_score)
#   print("Final_score", final_score)
#   print("Best_score", best_score)

In [ ]:
# # Softmax_v2
# from tqdm.auto import tqdm

# pb_train = tqdm(range(num_training_steps))
# pb_test = tqdm(range(num_epochs*len(test_dataloader)))
# best_score = -1

# for epoch in range(num_epochs):
#   train_loss = 0
#   val_loss = 0
#   model.train()
#   for batch in train_dataloader:
#     inputs = {'input_ids': batch['input_ids'].to(device),
#              'attention_mask': batch['attention_mask'].to(device)}
#     # outputs_classifier, outputs_regressor = model(**unknown_token(inputs, device))
#     outputs_regressor = model(**inputs)
#     # loss = nn.MSELoss()(outputs, batch['labels'].to(device).float())
#     # loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
#     # loss1 = bce_loss_weights(outputs_classifier, batch['labels_classifier'].to(device).float(), weights.to(device))
#     # loss1 = CB_loss(outputs_classifier, batch['labels_classifier'].to(device).float(), num_positive.to(device), num_negative.to(device))
    
#     loss2 = loss_softmax_v2(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
#     loss = loss2
#     optimizer.zero_grad()
#     # print(loss.item())
#     loss.backward()
#     optimizer.step()       
#     lr_scheduler.step()
#     pb_train.update(1)
#     pb_train.set_postfix(loss_regressor=loss2.item(),loss=loss.item())
#     train_loss += loss.item() / len(train_dataloader)
#   print("Train Loss:", train_loss)
#   model.eval()
#   val_loss = ScalarMetric()
#   val_loss_classifier = ScalarMetric()
#   val_loss_regressor = ScalarMetric()
#   val_acc = AccuracyMetric()
#   val_f1_score = F1_score()
#   val_r2_score = R2_score()
#   num = 0
#   correct = 0
#   result = None
#   model.eval()
#   for batch in test_dataloader:
#     inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#     with torch.no_grad():
#       outputs_regressor = model(**inputs)
#       # outputs = nn.Sigmoid()(outputs)
#       # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
#       loss2 = loss_softmax_v2(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
#       loss = loss2
#       outputs_regressor = outputs_regressor.cpu().numpy()
#       outputs_regressor = outputs_regressor.argmax(axis=-1)
#       y_true = batch['labels_regressor'].numpy()
#       # outputs = pred_to_label(outputs_classifier, outputs_regressor)
#       outputs = outputs_regressor
#       # y_true = prob_to_label(y_true)
#       # print(outputs)
#       result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#       val_loss_regressor.update(loss2.item())
#       val_loss.update(loss.item())
#       val_acc.update(np.round(outputs), y_true)
#       val_f1_score.update(np.round(outputs), y_true)
#       val_r2_score.update(np.round(outputs), y_true)
#       pb_test.update(1)
#   f1_score = val_f1_score.compute()
#   r2_score = val_r2_score.compute()
#   final_score = (f1_score * r2_score).sum()*1/6
#   if final_score > best_score:
#     best_score = final_score
#     torch.save(model.state_dict(), "weights/model_softmax_new_submit.pt")
#   print("Acc", val_acc.compute())
#   print("F1_score", f1_score)
#   print("R2_score", r2_score)
#   print("Final_score", final_score)
#   print("Best_score", best_score)

In [ ]:
# # Custom

# import numpy as np
# model = CustomModel("vinai/phobert-base", 30).to(device)
# model.load_state_dict(torch.load("weights/model_regress_v2_submit.pt"))
# val_loss = ScalarMetric()
# val_loss_classifier = ScalarMetric()
# val_loss_regressor = ScalarMetric()
# val_acc = AccuracyMetric()
# val_f1_score = F1_score()
# val_r2_score = R2_score()
# num = 0
# correct = 0
# result = None
# model.eval()
# for batch in test_dataloader:
#   inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#   with torch.no_grad():
#     outputs_classifier, outputs_regressor = model(**inputs)
#     # outputs = nn.Sigmoid()(outputs)
#     # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
#     loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
#     loss2 = loss_regressor(outputs_regressor, batch['labels_regressor'].to(device).float())
#     loss = loss1 + loss2
#     outputs_classifier = outputs_classifier.cpu().numpy()
#     outputs_regressor = outputs_regressor.cpu().numpy()
#     y_true = batch['labels_regressor'].numpy()
#     outputs = pred_to_label(outputs_classifier, outputs_regressor)
#     # y_true = prob_to_label(y_true)
#     # print(outputs)
#     result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#     val_loss_classifier.update(loss1.item())
#     val_loss_regressor.update(loss2.item())
#     val_loss.update(loss.item())
#     val_acc.update(np.round(outputs), y_true)
#     val_f1_score.update(np.round(outputs), y_true)
#     val_r2_score.update(np.round(outputs), y_true)
#     pb_train.update(1)
# f1_score = val_f1_score.compute()
# r2_score = val_r2_score.compute()
# final_score = (f1_score * r2_score).sum()*1/6
# print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
# print("Acc", val_acc.compute())
# print("F1_score", f1_score)
# print("R2_score", r2_score)
# print("Final_score", final_score)

In [ ]:
# # Softmax

# import numpy as np
# model = CustomModelSoftmax("vinai/phobert-base", 30).to(device)
# model.load_state_dict(torch.load("weights/model_softmax_v2_submit.pt"))
# val_loss = ScalarMetric()
# val_loss_classifier = ScalarMetric()
# val_loss_regressor = ScalarMetric()
# val_acc = AccuracyMetric()
# val_f1_score = F1_score()
# val_r2_score = R2_score()
# num = 0
# correct = 0
# result = None
# model.eval()
# for batch in test_dataloader:
#   inputs = {'input_ids': batch['input_ids'].to(device),
#             'attention_mask': batch['attention_mask'].to(device)}
#   with torch.no_grad():
#     outputs_classifier, outputs_regressor = model(**inputs)
#     # outputs = nn.Sigmoid()(outputs)
#     # loss = nn.BCELoss()(outputs, batch['labels'].to(device).float())
#     loss1 = loss_classifier(outputs_classifier, batch['labels_classifier'].to(device).float())
#     loss2 = loss_softmax(outputs_regressor, batch['labels_regressor'].to(device).float(), device)
#     loss = loss1 + loss2
#     outputs_classifier = outputs_classifier.cpu().numpy()
#     outputs_regressor = outputs_regressor.cpu().numpy()
#     outputs_regressor = outputs_regressor.argmax(axis=-1) + 1
#     y_true = batch['labels_regressor'].numpy()
#     outputs = pred_to_label(outputs_classifier, outputs_regressor)
#     # y_true = prob_to_label(y_true)
#     # print(outputs)
#     result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
#     val_loss_classifier.update(loss1.item())
#     val_loss_regressor.update(loss2.item())
#     val_loss.update(loss.item())
#     val_acc.update(np.round(outputs), y_true)
#     val_f1_score.update(np.round(outputs), y_true)
#     val_r2_score.update(np.round(outputs), y_true)
# f1_score = val_f1_score.compute()
# r2_score = val_r2_score.compute()
# final_score = (f1_score * r2_score).sum()*1/6
# print("Test Loss:", val_loss.compute(), "Loss Classifier:", val_loss_classifier.compute(), "Loss Regressor:", val_loss_regressor.compute())
# print("Acc", val_acc.compute())
# print("F1_score", f1_score)
# print("R2_score", r2_score)
# print("Final_score", final_score)

In [ ]:
import numpy as np

def score_to_tensor(score):
  tensor = np.zeros((score.shape[0], 6, 6))
  mask_up = np.ceil(score).reshape(-1).astype(np.int16)
  mask_down = np.floor(score).reshape(-1).astype(np.int16)
  xv, yv = np.meshgrid(np.arange(score.shape[0]), np.arange(6))
  y = yv.T.reshape(-1).astype(np.int16)
  x = xv.T.reshape(-1).astype(np.int16)
  score_up = (score - np.floor(score)).reshape(-1)
  score_down = (1 - score_up).reshape(-1)
  tensor[x, y, mask_up] = score_up
  tensor[x, y, mask_down] = score_down
  tensor[:,:,1] = tensor[:,:,0] + tensor[:,:,1]
  return tensor[:,:,1:]


In [ ]:
# Ensemble

import numpy as np
device = "cuda" if torch.cuda.is_available() else "cpu"
model1 = CustomModelSoftmax("vinai/phobert-base", 30).to(device)
model2 = CustomModel("vinai/phobert-base", 30).to(device)
model3 = CustomModelMultiHead("vinai/phobert-base", 30).to(device)
# model4 = CustomModelSoftmax_v2("vinai/phobert-base", 30).to(device)
model4 = CustomModelMultiHeadRegressor("vinai/phobert-base", 30).to(device)
# model3 = CustomModelLabelAttention("vinai/phobert-base", 30).to(device)
model1.load_state_dict(torch.load("weights/model_softmax_final_submit.pt"))
model2.load_state_dict(torch.load("weights/model_regress_final_submit.pt"))
model3.load_state_dict(torch.load("weights/model_attention_softmax_final_submit.pt"))
model4.load_state_dict(torch.load("weights/model_attention_regress_final_submit.pt"))
val_loss = ScalarMetric()
val_loss_classifier = ScalarMetric()
val_loss_regressor = ScalarMetric()
val_acc = AccuracyMetric()
val_f1_score = F1_score()
val_r2_score = R2_score()
num = 0
correct = 0
result = None
model1.eval()
model2.eval()
model3.eval()
model4.eval()
# model4.eval()
for batch in test_dataloader:
  inputs = {'input_ids': batch['input_ids'].to(device),
            'attention_mask': batch['attention_mask'].to(device)}
  with torch.no_grad():
    outputs_classifier1, outputs_regressor1 = model1(**inputs)
    outputs_classifier2, outputs_regressor2 = model2(**inputs)
    outputs_classifier3, outputs_regressor3 = model3(**inputs)
    outputs_classifier4, outputs_regressor4 = model4(**inputs)

    outputs_classifier1 = outputs_classifier1.cpu().numpy()
    outputs_regressor1 = outputs_regressor1.cpu().numpy()
    outputs_classifier2 = outputs_classifier2.cpu().numpy()
    outputs_regressor2 = outputs_regressor2.cpu().numpy()
    outputs_classifier3 = outputs_classifier3.cpu().numpy()
    outputs_regressor3 = outputs_regressor3.cpu().numpy()
    outputs_classifier4 = outputs_classifier4.cpu().numpy()
    outputs_regressor4 = outputs_regressor4.cpu().numpy()

    outputs_regressor2 = score_to_tensor(outputs_regressor2)
    outputs_regressor4 = score_to_tensor(outputs_regressor4)

    outputs_regressor = (nn.Softmax(dim=-1)(torch.tensor(outputs_regressor1)).numpy() + outputs_regressor2 + nn.Softmax(dim=-1)(torch.tensor(outputs_regressor3)).numpy() + outputs_regressor4)/4
    outputs_classifier = (outputs_classifier1 + outputs_classifier2 + outputs_classifier3 + outputs_classifier4)/4

    # outputs_regressor = (nn.Softmax(dim=-1)(torch.tensor(outputs_regressor1)).numpy() + outputs_regressor2)/2
    # outputs_classifier = (outputs_classifier1 + outputs_classifier2)/2

    outputs_regressor = outputs_regressor.argmax(axis=-1) + 1

    y_true = batch['labels_regressor'].numpy()
    outputs = pred_to_label(outputs_classifier, outputs_regressor)
    # y_true = prob_to_label(y_true)
    # print(outputs)
    result = np.concatenate([result, np.round(outputs)], axis=0) if result is not None else np.round(outputs)
    val_acc.update(np.round(outputs), y_true)
    val_f1_score.update(np.round(outputs), y_true)
    val_r2_score.update(np.round(outputs), y_true)
f1_score = val_f1_score.compute()
r2_score = val_r2_score.compute()
final_score = (f1_score * r2_score).sum()*1/6
print("Acc", val_acc.compute())
print("F1_score", f1_score)
print("R2_score", r2_score)
print("Final_score", final_score)

Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaModel: ['lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.

Precision: [0.67647059 0.93478261 0.6969697  0.90232558 0.89108911 0.76744186]
Recall [0.61333333 0.84313725 0.575      0.92822967 0.86538462 0.78571429]
Acc 0.8853333333333333
F1_score [0.64335664 0.88659794 0.63013699 0.91509434 0.87804878 0.77647059]
R2_score [0.9701087  0.97020349 0.98097826 0.96617268 0.96736111 0.94886364]
Final_score 0.7621251786927838


In [ ]:
# import pandas as pd
# test_data = pd.read_csv("data/test_data_submit.csv")
# feature = ["giai_tri", "luu_tru", "nha_hang", "an_uong", "di_chuyen", "mua_sam"]
# for i in range(6):
#   test_data[f"pred_{feature[i]}"] = result[:, i]

In [ ]:
# test_data = test_data.drop(["Unnamed: 0"], axis=1)

In [ ]:
# mask_a = test_data.iloc[:, 1:7] != 0
# mask_b = test_data.iloc[:, 7:] != 0
# error_aspect = np.logical_xor(mask_a, mask_b)
# data_error_aspect = test_data[np.any(error_aspect, axis=1)]
# error_type = []
# for idx in range(len(data_error_aspect)):
#   error = ""
#   for i in range(6):
#     aspect = feature[i]
#     if (data_error_aspect.iloc[idx][i + 1] != 0) != (data_error_aspect.iloc[idx][i + 7] != 0):
#       error += (feature[i] + ' ')
#   error_type.append(error)
# data_error_aspect['error_type'] = error_type
# data_error_aspect.to_csv('error_aspect_submit.csv', index=False)

In [ ]:
# data_error_aspect.to_csv('error_aspect_submit.csv', encoding="utf-8",index=False)

In [ ]:
# data_true_aspect = test_data[np.logical_not(np.any(error_aspect, axis=1))]
# error_rate = []
# for idx in range(len(data_true_aspect)):
#   error = ""
#   for i in range(6):
#     aspect = feature[i]
#     if data_true_aspect.iloc[idx][i + 1] != data_true_aspect.iloc[idx][i + 7]:
#       error += (feature[i] + f"({data_true_aspect.iloc[idx][i + 1], data_true_aspect.iloc[idx][i + 7]})" +',')
#   error_rate.append(error)
# data_true_aspect['error_rate'] = error_rate
# data_true_aspect.to_csv('error_rate_submit.csv', index=False, encoding="utf-8")

In [ ]:
from google.colab import output
output.disable_custom_widget_manager()

In [ ]:
%ls weights

model_attention_regress_final_submit.pt  model_regress_full_submit.pt
model_attention_regress_full_submit.pt   model_regress_relabel_submit.pt
model_attention_regress_submit.pt        model_regress_v2_submit.pt
model_attention_softmax_final_submit.pt  model_softmax_add_nha_hang_submit.pt
model_attention_softmax_full_submit.pt   model_softmax_final_submit.pt
model_attention_softmax_submit.pt        model_softmax_full_submit.pt
model_label_softmax_submit.pt            model_softmax_new_submit.pt
model_label_v2_submit.pt                 model_softmax_relabel_submit.pt
model_regress_add_nha_hang_submit.pt     model_softmax_v2_submit.pt
model_regress_final_submit.pt
